# The effects of studying on high school students

**Authors:** Nadav Tadelis, Sarah Johnson, Chitwan Kaudan

## Abstract

As students, a large part of our daily life is taken up by studying. Currently we must make informed guesses about how much an extra hour of studying will help our grades. However this is not ideal; when making allocation decisions, imperfect information can become a major issue. Specifically, if imperfect information causes me to incorrectly model the effect of studying on grades, then we make poor decisions about how much of my time we should allocate to studying (effectively resulting in a loss of utility). If we were able to find the 'true' marginal effect of studying on grades, then we could calibrate our inner models in a more accurate way. 

Using 

## Exploring the effects of studying



The first step is to build a model and make some assumptions to define the relationship between grades and studying. Let an individual's grade be $\mathrm{G}_i$ and weekly hours of studying be $\mathrm{S}_i$ and their "ability" be $\mathrm{A}_i$. Then we can write:
$$
\mathrm{G}_i = \beta_0 + \beta_1 \mathrm{S}_i + \beta_2 \mathrm{A}_i +\mathrm{U}_i
$$
Where $\mathrm{U}_i$ captures some unobserved heterogeneity and disturbances. 

Clearly, there are issues with this model. How are we defining grades? The ideal set up would have a course specific system of equations, where the number of equations is equal to the number of courses. The next best set up would involve estimating one equation for each type of course (quantitative, literary, historical, etc.). Another alternative would be to define grades as cummulative GPA. In this paper, due to the limiations of the data, we will only be estimating one of the equations described in the second set up; we will define $\mathrm{G}_i$ as the final grade in a Portuguese class. 

Another issue with this model is: how are we defining ability? By its very nature, ability is unobserved. We can proxy for ability using other individual level characteristics (intelligence, age, parent's education, etc.) however we cannot fully capture ability because it is hard to fully define it. Hence, we must keep in mind that the model is not going to be fully specified.

We can think of our utility as being some function of grades, free time (let's consider everything that is not studying, sleeping, or class as "free time"), and studying time. We would expect that the coefficients on grades and free time would be positive, and the coefficient on studying time to be negative; with magnitudes of these coefficients being determined by an individual's preferences. For example, somebody who cares very little about grades, enjoys constantly partying, and hates studying, would have a small positive coefficient on grades, a large positive coefficient on free time, and a large negative coefficient on studying time$^1$. This person would maximize their utility and choose how to allocate their time, and they would probably end up spending very little time studying. Notice that before maximizing their utility, an individual would replace grades with the previously defined model for grades (dependent on studying and ability); so someone who heavily values grades could end up having a positive coefficient on studying after plugging the model for grades into their utility function.

Establishing this utility function gives motivation for including variables that might introduce multicollinearity. For example, weekly amount of free time might not improve our estimate of the marginal effect of studying, and would be collinear with the amount of weekly studying. However, when we think of our observations as realizations of a decision making process that involves utility maximization, we can see that there is a strong argument for including free time in the final model.

-----
1. Of course, for some people and some subject matters of study, the coefficient on studying time may be positive with a decreasing marginal utility. We simplify the specification here dramatically.

The [data](https://archive.ics.uci.edu/ml/datasets/Student+Performance# "Student Performance Data Set") being used are from the public archive of UCI's machine learning repository and were collected by Paulo Cortez of the University of Minho, Portugal in the 2005 - 2006 academic year. 

with open('results/2SLS.pkl','rb') as f:
    model_results = pickle.load(f)

## Conclusions

After addressing our issues of endogeneity, we end up with a model that is plausibly identifying the marginal effect of studying on grades. Our results estimate that a marginal increase in hours of studying (per week) results in a 36 point increase in final grade (as a percent). We must keep in mind the limitations of this model; the limited sample greatly reduce our ability to make general statements about this finding. However, our results are still indicative that in secondary schools, additional studying causes significant increases to final grades. 

A direction for future study would include an increase to the complexity of the model. It seems unlikely that three hours of studying per week is be enough to ensure almost anyone gets a perfect grade - so perhaps we need to allow for a quadratic (or higher degree) relationship between studying and grades.

### References

Card, D., & Krueger, A. (1992). Does School Quality Matter? Returns to Education and the Characteristics of Public Schools in the United States. *Journal of Political Economy*, 100(1), 1-40.

Cortez, P. and Silva, A. Using Data Mining to Predict Secondary School Student Performance. In A. Brito and J. Teixeira Eds., Proceedings of 5th FUture BUsiness TEChnology Conference (FUBUTEC 2008) pp. 5-12, Porto, Portugal, April, 2008, EUROSIS, ISBN 978-9077381-39-7. 

Greene, W. H. (2000). Econometric analysis. Upper Saddle River, N.J: Prentice Hall.

MacKinnon, J.G. and H. White. (1985), Some heteroskedasticity consistent covariance matrix estimators with improved finite sample properties. *Journal of Econometrics*, 29, 53-57.

### Author Contributions
